# Generating High Rez GAN Faces with Google CoLab

This notebook demonstrates how to run [NVidia StyleGAN](https://github.com/NVlabs/stylegan) inside of a Google CoLab notebook.  I suggest you use this to generate GAN faces from a pretrained model.  If you try to train your own, you will run into compute limitations of Google CoLab.

Make sure to run this code on a GPU instance.  GPU is assumed.

This is from my [class on deep learning](  https://www.youtube.com/watch?v=EQ38k6z2aks&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN)

[Jeff Heaton](https://www.heatonresearch.com/)

# Instructions

First, map your G-Drive, this is where your GANs will be written to.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Next, clone Stylegan from GitHub.

In [ ]:
!git clone https://github.com/NVlabs/stylegan.git

Cloning into 'stylegan'...
remote: Enumerating objects: 89, done.
remote: Total 89 (delta 0), reused 0 (delta 0), pack-reused 89
Receiving objects: 100% (89/89), 1.68 MiB | 14.59 MiB/s, done.
Resolving deltas: 100% (33/33), done.


Verify that Stylegan has been cloned.

In [ ]:
!ls /content/stylegan/

config.py	     LICENSE.txt	    run_metrics.py
dataset_tool.py      metrics		    stylegan-teaser.png
dnnlib		     pretrained_example.py  training
generate_figures.py  README.md		    train.py


Add the Stylegan folder to Python so that you can import it.

In [ ]:
import sys
sys.path.insert(0, "/content/stylegan")
!pip install https://github.com/podgorskiy/dnnlib/releases/download/0.0.1/dnnlib-0.0.1-py3-none-any.whl


In [ ]:
import dnnlib

In [ ]:
!git clone https://github.com/mmuhammetcavus/dnnlib.git

Cloning into 'dnnlib'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 24 (delta 3), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (24/24), 43.26 KiB | 3.09 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [ ]:
# !git install https://github.com/mmuhammetcavus/dnnlib.git!
!git --help

usage: git [--version] [--help] [-C <path>] [-c <name>=<value>]
           [--exec-path[=<path>]] [--html-path] [--man-path] [--info-path]
           [-p | --paginate | -P | --no-pager] [--no-replace-objects] [--bare]
           [--git-dir=<path>] [--work-tree=<path>] [--namespace=<name>]
           [--super-prefix=<path>] [--config-env=<name>=<envvar>]
           <command> [<args>]

These are common Git commands used in various situations:

start a working area (see also: git help tutorial)
   clone     Clone a repository into a new directory
   init      Create an empty Git repository or reinitialize an existing one

work on the current change (see also: git help everyday)
   add       Add file contents to the index
   mv        Move or rename a file, a directory, or a symlink
   restore   Restore working tree files
   rm        Remove files from the working tree and from the index

examine the history and state (see also: git help revisions)
   bisect    Use binary search to find th

In [ ]:
!pip install tflib

  Preparing metadata (setup.py) ... done
  Created wheel for tflib: filename=tflib-0.1.0-py3-none-any.whl size=1818 sha256=2ad367ab9fc9d34035bb7531c6016a25714ab4b1626d46832ef19bfe45b663df
  Stored in directory: /root/.cache/pip/wheels/95/fd/d2/42015053fc7059e28c1b430322e583c459d65f805bfb2f90e6
Successfully built tflib


In [ ]:
!ls dnnlib/  drive/  sample_data/  stylegan/

dnnlib/:
__init__.py  README.md	tflib  util.py

drive/:
MyDrive  Shareddrives

sample_data/:
anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md

stylegan/:
config.py	     LICENSE.txt	    README.md		 train.py
dataset_tool.py      metrics		    run_metrics.py
dnnlib		     pretrained_example.py  stylegan-teaser.png
generate_figures.py  __pycache__	    training


The code below is baed on code from NVidia.  This actually generates your images.

In [ ]:
import tflib

ModuleNotFoundError: ignored

In [ ]:
# Copyright (c) 2019, NVIDIA CORPORATION. All rights reserved.
#
# This work is licensed under the Creative Commons Attribution-NonCommercial
# 4.0 International License. To view a copy of this license, visit
# http://creativecommons.org/licenses/by-nc/4.0/ or send a letter to
# Creative Commons, PO Box 1866, Mountain View, CA 94042, USA.

"""Minimal script for generating an image using pre-trained StyleGAN generator."""

import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import tflib
# import dnnlib.tflib
import config

def main():
    # Initialize TensorFlow.
    tflib.init_tf()

    # Load pre-trained network.
    url = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ' # karras2019stylegan-ffhq-1024x1024.pkl
    with dnnlib.util.open_url(url, cache_dir=config.cache_dir) as f:
        _G, _D, Gs = pickle.load(f)
        # _G = Instantaneous snapshot of the generator. Mainly useful for resuming a previous training run.
        # _D = Instantaneous snapshot of the discriminator. Mainly useful for resuming a previous training run.
        # Gs = Long-term average of the generator. Yields higher-quality results than the instantaneous snapshot.

    # Print network details.
    Gs.print_layers()

    # Pick latent vector.
    rnd = np.random.RandomState()


    latents = rnd.randn(1, Gs.input_shape[1])

    # Generate image.
    fmt = dict(func=dnnlib.tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    images = Gs.run(latents, None, truncation_psi=0.7, randomize_noise=True, output_transform=fmt)

    # Save image.
    os.makedirs(config.result_dir, exist_ok=True)
    png_filename = os.path.join(config.result_dir, f'/content/drive/My Drive/images/example1.png')
    PIL.Image.fromarray(images[0], 'RGB').save(png_filename)

if __name__ == "__main__":
    main()


AttributeError: ignored

In [ ]:
!ls /content/drive/My\ Drive/

'Colab Notebooks'				  example.png
'Copy of karras2019stylegan-ffhq-1024x1024.pkl'   projects
